In [1]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Fraudfinder - Feature Engineering (batch)

<table align="left">
  <td>
    <a href="https://console.cloud.google.com/ai-platform/notebooks/deploy-notebook?&download_url=https://github.com/GoogleCloudPlatform/fraudfinder/raw/main/02_feature_engineering_batch.ipynb">
       <img src="https://www.gstatic.com/cloud/images/navigation/vertex-ai.svg" alt="Google Cloud Notebooks">Open in Cloud Notebook
    </a>
  </td> 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/fraudfinder/blob/main/02_feature_engineering_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Open in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/fraudfinder/blob/main/02_feature_engineering_batch.ipynb">
        <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

## Overview

[Fraudfinder](https://github.com/googlecloudplatform/fraudfinder) is a series of labs on how to build a real-time fraud detection system on Google Cloud. Throughout the Fraudfinder labs, you will learn how to read historical bank transaction data stored in data warehouse, read from a live stream of new transactions, perform exploratory data analysis (EDA), do feature engineering, ingest features into a feature store, train a model using feature store, register your model in a model registry, evaluate your model, deploy your model to an endpoint, do real-time inference on your model with feature store, and monitor your model.

### Objective

As you engineer features for model training, it's important to consider how the features are computed when making predictions with new data. For online predictions, you may have features that can be pre-computed via _batch feature engineering_. You may also features that need to be computed on-the-fly via _streaming-based feature engineering_. For these Fraudfinder labs, for computing features based on the last n _days_, you will use _batch_ feature engineering in BigQuery; for computing features based on the last n _minutes_, you will use _streaming-based_ feature engineering using Dataflow.

This notebook shows how to generate new features on bank transactions by customer and terminal over the last n days, by doing batch feature engineering in SQL with BigQuery. Then, you will create a feature store using Vertex AI Feature Store, and ingest your newly-created features from BigQuery into Vertex AI Feature Store, so that a feature store can become the single source of data for both training and model inference. 

You will also create some placeholder values for streaming-based feature engineering, which is covered in the next notebook, `03_feature_engineering_streaming.ipynb`.

This lab uses the following Google Cloud services and resources:

- [Vertex AI](https://cloud.google.com/vertex-ai/)
- [BigQuery](https://cloud.google.com/bigquery/)


Steps performed in this notebook:

- Build customer and terminal-related features
- Create Feature store, entities and features
- Ingest feature values in Feature store from BigQuery table
- Read features from the feature store

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Load configuration settings from the setup notebook

Set the constants used in this notebook and load the config settings from the `00_environment_setup.ipynb` notebook.

In [3]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
BUCKET_NAME = f"{PROJECT_ID}-fraudfinder"
config = !gsutil cat gs://{BUCKET_NAME}/config/notebook_env.py
print(config.n)
exec(config.n)


BUCKET_NAME          = "dryrun01-fraudfinder"
PROJECT              = "dryrun01"
REGION               = "us-central1"
ID                   = "9selj"
FEATURESTORE_ID      = "fraudfinder_9selj"
MODEL_NAME           = "fraudfinder_logreg_model"
ENDPOINT_NAME        = "fraudfinder_logreg_endpoint"
TRAINING_DS_SIZE     = "1000"



### Import libraries

In [4]:
# General
import datetime as dt
import json
import os
import random
import sys
import time
from datetime import datetime, timedelta
from typing import List, Union

# Data Engineering
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 500)

# Vertex AI and Vertex AI Feature Store
from google.cloud import aiplatform as vertex_ai
from google.cloud import bigquery
from google.cloud.aiplatform import EntityType, Feature, Featurestore

### Define constants

In [5]:
# Define the date range of transactions for feature engineering (last 10 days up until yesterday)
YESTERDAY = datetime.today() - timedelta(days=1)
YEAR_MONTH_PREFIX = YESTERDAY.strftime("%Y-%m")
DATAPROCESSING_START_DATE = (YESTERDAY - timedelta(days=10)).strftime("%Y-%m-%d")
DATAPROCESSING_END_DATE = YESTERDAY.strftime("%Y-%m-%d")

# Define BiqQuery dataset and tables to calculate features.
RAW_BQ_TRANSACTION_TABLE_URI = f"{PROJECT_ID}.tx.tx"
RAW_BQ_LABELS_TABLE_URI = f"{PROJECT_ID}.tx.txlabels"
FEATURES_BQ_TABLE_URI = f"{PROJECT_ID}.tx.wide_features_table"

# Define Vertex AI Feature store settings.
CUSTOMERS_TABLE_NAME = f"customers_{DATAPROCESSING_END_DATE.replace('-', '')}"
CUSTOMERS_BQ_TABLE_URI = f"{PROJECT_ID}.tx.{CUSTOMERS_TABLE_NAME}"
TERMINALS_TABLE_NAME = f"terminals_{DATAPROCESSING_END_DATE.replace('-', '')}"                    
TERMINALS_BQ_TABLE_URI = f"{PROJECT_ID}.tx.{TERMINALS_TABLE_NAME}"
ONLINE_STORAGE_NODES = 1
FEATURE_TIME = "feature_ts"
CUSTOMER_ENTITY_ID = "customer"
TERMINAL_ENTITY_ID = "terminal"

### Helpers

Define a set of helper functions to run BigQuery query and create features. 

In [6]:
def run_bq_query(sql: str, show=False) -> Union[str, pd.DataFrame]:
    """
    Run a BigQuery query and return the job ID or result as a DataFrame
    Args:
        sql: SQL query, as a string, to execute in BigQuery
        show: A flag to show query result in a Pandas Dataframe
    Returns:
        df: DataFrame of results from query,  or error, if any
    """

    bq_client = bigquery.Client()

    # Try dry run before executing query to catch any errors
    job_config = bigquery.QueryJobConfig(dry_run=True, use_query_cache=False)
    bq_client.query(sql, job_config=job_config)

    # If dry run succeeds without errors, proceed to run query
    job_config = bigquery.QueryJobConfig()
    client_result = bq_client.query(sql, job_config=job_config)

    job_id = client_result.job_id

    # Wait for query/job to finish running. then get & return data frame
    result = client_result.result()
    print(f"Finished job_id: {job_id}")
    
    if show:
        df = result.to_arrow().to_pandas()
        return df

## Feature Engineering

### Define customer and terminal-related features for batch feature engineering

In this section, you will create features, based on historical customer behaviour and historical terminal activities. This features will be batch-generated using SQL in BigQuery, where the historical data is stored.

The query below will calculate 2 sets of features: 

1. **Customer-related features**: which describes the spending behaviour of customer within 1, 7 and 15 days time windows using number of transactions and average amount spent in dollars ($)

2. **Terminal-related features** which describes the risk of a given terminal to be exposed to fraudulent transactions within 1, 7 and 15 days using average number of fraudulent transactions in dollars ($), the number of transactions and risk index. One thing to note is that you will add some delay to take into account time that would pass between the time of transaction and the result of fraud investigation or customer claim.

You will use one month of transaction data starting from the end of January and going back to compute the features.

Below is the schema you should expect to see, after doing the batch feature engineering in BigQuery:

|feature_time           |customer_id| customer batch features   |
|-----------------------|-----------|---------------------------|
|2022-01-01 17:20:15 UTC|1          |(e.g., nb_tx,  avg_tx)     |
|2022-01-02 12:08:40 UTC|2          |(e.g., nb_tx,  avg_tx)     |
|2022-01-03 17:30:48 UTC|3          |(e.g., nb_tx,  avg_tx)     |


|feature_time           |terminal_id| terminal batch features|
|-----------------------|-----------|------------------------|
|2022-01-01 17:20:15 UTC|12345      |(e.g., risk_x_days)     |
|2022-01-02 12:08:40 UTC|26789      |(e.g., risk_x_days)     |
|2022-01-03 17:30:48 UTC|101112     |(e.g., risk_x_days)     |


#### Create the query to create batch features

Date settings to be used:

In [7]:
print(f"""
DATAPROCESSING_START_DATE: {DATAPROCESSING_START_DATE}
DATAPROCESSING_END_DATE: {DATAPROCESSING_END_DATE}
""")


DATAPROCESSING_START_DATE: 2023-01-14
DATAPROCESSING_END_DATE: 2023-01-24



##### Customer feature table

Customer table SQL query string:

In [8]:
create_customer_batch_features_query = f"""
CREATE OR REPLACE TABLE {CUSTOMERS_BQ_TABLE_URI} AS
WITH
  -- query to join labels with features -------------------------------------------------------------------------------------------
  get_raw_table AS (
  SELECT
    raw_tx.TX_TS,
    raw_tx.TX_ID,
    raw_tx.CUSTOMER_ID,
    raw_tx.TERMINAL_ID,
    raw_tx.TX_AMOUNT,
    raw_lb.TX_FRAUD
  FROM (
    SELECT
      *
    FROM
      `{RAW_BQ_TRANSACTION_TABLE_URI}`
    WHERE
      DATE(TX_TS) BETWEEN DATE_SUB("{DATAPROCESSING_END_DATE}", INTERVAL 15 DAY) AND "{DATAPROCESSING_END_DATE}"
    ) raw_tx
  LEFT JOIN 
    `{RAW_BQ_LABELS_TABLE_URI}` as raw_lb
  ON raw_tx.TX_ID = raw_lb.TX_ID),

  -- query to calculate CUSTOMER spending behaviour --------------------------------------------------------------------------------
  get_customer_spending_behaviour AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    TX_AMOUNT,
    TX_FRAUD,
    
    # calc the number of customer tx over daily windows per customer (1, 7 and 15 days, expressed in seconds)
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 86400 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_1DAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_7DAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_NB_TX_14DAY_WINDOW,
      
    # calc the customer average tx amount over daily windows per customer (1, 7 and 15 days, expressed in seconds, in dollars ($))
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 86400 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW,
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW,
    AVG(TX_AMOUNT) OVER (PARTITION BY CUSTOMER_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS CUSTOMER_ID_AVG_AMOUNT_14DAY_WINDOW,
  FROM get_raw_table)

# Create the table with CUSTOMER and TERMINAL features ----------------------------------------------------------------------------
SELECT
  PARSE_TIMESTAMP("%Y-%m-%d %H:%M:%S", FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TX_TS, "UTC")) AS feature_ts,
  CUSTOMER_ID AS customer_id,
  CAST(CUSTOMER_ID_NB_TX_1DAY_WINDOW AS INT64) AS customer_id_nb_tx_1day_window,
  CAST(CUSTOMER_ID_NB_TX_7DAY_WINDOW AS INT64) AS customer_id_nb_tx_7day_window,
  CAST(CUSTOMER_ID_NB_TX_14DAY_WINDOW AS INT64) AS customer_id_nb_tx_14day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_1DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_1day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_7DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_7day_window,
  CAST(CUSTOMER_ID_AVG_AMOUNT_14DAY_WINDOW AS FLOAT64) AS customer_id_avg_amount_14day_window,
FROM
  get_customer_spending_behaviour
"""

##### Run the query 

You create the customer features table

In [9]:
run_bq_query(create_customer_batch_features_query)

Finished job_id: 1dadf1e0-ff6a-46c5-9bfa-a0bbf869914c


##### Inspect the result 

You can query some data rows to validate the result of the query

In [10]:
run_bq_query(
    f"SELECT * FROM `{CUSTOMERS_BQ_TABLE_URI}` LIMIT 10",
    show=True
)

Finished job_id: e0ff728b-dcb9-479d-9ff2-e8f1c8104f40


,feature_ts,customer_id,customer_id_nb_tx_1day_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_1day_window,customer_id_avg_amount_7day_window,customer_id_avg_amount_14day_window
0,2023-01-10 19:35:35+00:00,0013855949975740,8,8,8,31.947500,31.947500,31.947500
1,2023-01-17 19:00:26+00:00,0014775767750327,2,11,11,330.835000,206.519091,206.519091
2,2023-01-13 20:43:57+00:00,0025127990883783,1,3,3,29.180000,33.780000,33.780000
3,2023-01-17 17:22:50+00:00,0152324682921351,1,10,11,38.400000,38.090000,38.300909
4,2023-01-22 18:26:21+00:00,0152324682921351,1,9,16,33.030000,37.378889,38.491875
5,2023-01-19 11:23:29+00:00,0176450580389874,8,26,34,68.246250,71.874231,72.153235
6,2023-01-20 23:20:33+00:00,0212344466701883,2,11,17,31.535000,39.148182,39.248235
7,2023-01-23 16:19:46+00:00,0217626316883302,4,22,40,39.305000,36.217273,34.965000
8,2023-01-11 09:40:41+00:00,0234378776179099,5,10,10,50.742000,51.700000,51.700000
9,2023-01-21 14:29:54+00:00,0234378776179099,3,13,35,50.266667,52.179231,53.111143


##### Terminal feature table

Terminal table SQL query string:

In [11]:
create_terminal_batch_features_query = f"""
CREATE OR REPLACE TABLE {TERMINALS_BQ_TABLE_URI} AS
WITH
  -- query to join labels with features -------------------------------------------------------------------------------------------
  get_raw_table AS (
  SELECT
    raw_tx.TX_TS,
    raw_tx.TX_ID,
    raw_tx.CUSTOMER_ID,
    raw_tx.TERMINAL_ID,
    raw_tx.TX_AMOUNT,
    raw_lb.TX_FRAUD
  FROM (
    SELECT
      *
    FROM
      `{RAW_BQ_TRANSACTION_TABLE_URI}`
    WHERE
      DATE(TX_TS) BETWEEN DATE_SUB("{DATAPROCESSING_END_DATE}", INTERVAL 15 DAY) AND "{DATAPROCESSING_END_DATE}"
    ) raw_tx
  LEFT JOIN 
    `{RAW_BQ_LABELS_TABLE_URI}` as raw_lb
  ON raw_tx.TX_ID = raw_lb.TX_ID),

  # query to calculate TERMINAL spending behaviour --------------------------------------------------------------------------------
  get_variables_delay_window AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    
    # calc total amount of fraudulent tx and the total number of tx over the delay period per terminal (7 days - delay, expressed in seconds)
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_DELAY,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 604800 PRECEDING
      AND CURRENT ROW ) AS NB_TX_DELAY,
      
    # calc total amount of fraudulent tx and the total number of tx over the delayed window per terminal (window + 7 days - delay, expressed in seconds)
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 691200 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_1_DELAY_WINDOW,
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_7_DELAY_WINDOW,
    SUM(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1814400 PRECEDING
      AND CURRENT ROW ) AS NB_FRAUD_14_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 691200 PRECEDING
      AND CURRENT ROW ) AS NB_TX_1_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1209600 PRECEDING
      AND CURRENT ROW ) AS NB_TX_7_DELAY_WINDOW,
    COUNT(TX_FRAUD) OVER (PARTITION BY TERMINAL_ID ORDER BY UNIX_SECONDS(TX_TS) ASC RANGE BETWEEN 1814400 PRECEDING
      AND CURRENT ROW ) AS NB_TX_14_DELAY_WINDOW,
  FROM get_raw_table),

  # query to calculate TERMINAL risk factors ---------------------------------------------------------------------------------------
  get_risk_factors AS (
  SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    # calculate numerator of risk index
    NB_FRAUD_1_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_1DAY_WINDOW,
    NB_FRAUD_7_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_7DAY_WINDOW,
    NB_FRAUD_14_DELAY_WINDOW - NB_FRAUD_DELAY AS TERMINAL_ID_NB_FRAUD_14DAY_WINDOW,
    # calculate denominator of risk index
    NB_TX_1_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_1DAY_WINDOW,
    NB_TX_7_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_7DAY_WINDOW,
    NB_TX_14_DELAY_WINDOW - NB_TX_DELAY AS TERMINAL_ID_NB_TX_14DAY_WINDOW,
      FROM
    get_variables_delay_window),

  # query to calculate the TERMINAL risk index -------------------------------------------------------------------------------------
  get_risk_index AS (
    SELECT
    TX_TS,
    TX_ID,
    CUSTOMER_ID,
    TERMINAL_ID,
    TERMINAL_ID_NB_TX_1DAY_WINDOW,
    TERMINAL_ID_NB_TX_7DAY_WINDOW,
    TERMINAL_ID_NB_TX_14DAY_WINDOW,
    # calculate the risk index
    (TERMINAL_ID_NB_FRAUD_1DAY_WINDOW/(TERMINAL_ID_NB_TX_1DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_1DAY_WINDOW,
    (TERMINAL_ID_NB_FRAUD_7DAY_WINDOW/(TERMINAL_ID_NB_TX_7DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_7DAY_WINDOW,
    (TERMINAL_ID_NB_FRAUD_14DAY_WINDOW/(TERMINAL_ID_NB_TX_14DAY_WINDOW+0.0001)) AS TERMINAL_ID_RISK_14DAY_WINDOW
    FROM get_risk_factors 
  )

# Create the table with CUSTOMER and TERMINAL features ----------------------------------------------------------------------------
SELECT
  PARSE_TIMESTAMP("%Y-%m-%d %H:%M:%S", FORMAT_TIMESTAMP("%Y-%m-%d %H:%M:%S", TX_TS, "UTC")) AS feature_ts,
  TERMINAL_ID AS terminal_id,
  CAST(TERMINAL_ID_NB_TX_1DAY_WINDOW AS INT64) AS terminal_id_nb_tx_1day_window,
  CAST(TERMINAL_ID_NB_TX_7DAY_WINDOW AS INT64) AS terminal_id_nb_tx_7day_window,
  CAST(TERMINAL_ID_NB_TX_14DAY_WINDOW AS INT64) AS terminal_id_nb_tx_14day_window,
  CAST(TERMINAL_ID_RISK_1DAY_WINDOW AS FLOAT64) AS terminal_id_risk_1day_window,
  CAST(TERMINAL_ID_RISK_7DAY_WINDOW AS FLOAT64) AS terminal_id_risk_7day_window,
  CAST(TERMINAL_ID_RISK_14DAY_WINDOW AS FLOAT64) AS terminal_id_risk_14day_window,
FROM
  get_risk_index
"""

##### Run the query 

You create the customer features table

In [12]:
run_bq_query(create_terminal_batch_features_query)

Finished job_id: 5ba3ec19-942b-4872-a20d-dc622838f108


##### Inspect the result 

You can query some data rows to validate the result of the query

In [13]:
run_bq_query(
    f"SELECT * FROM `{TERMINALS_BQ_TABLE_URI}` LIMIT 10",
    show=True
)

Finished job_id: 3314a619-3650-4a94-b9b9-503068e83724


,feature_ts,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_7day_window,terminal_id_nb_tx_14day_window,terminal_id_risk_1day_window,terminal_id_risk_7day_window,terminal_id_risk_14day_window
0,2023-01-09 04:44:34+00:00,00150363,0,0,0,0.000000,0.000000,0.000000
1,2023-01-13 17:19:04+00:00,00150363,0,0,0,0.000000,0.000000,0.000000
2,2023-01-18 23:02:17+00:00,00150363,18,52,52,0.000000,0.000000,0.000000
3,2023-01-19 02:32:06+00:00,00150363,17,54,54,0.000000,0.000000,0.000000
4,2023-01-19 08:39:39+00:00,00150363,20,61,61,0.000000,0.000000,0.000000
5,2023-01-22 04:05:32+00:00,00150363,20,118,118,0.000000,0.000000,0.000000
6,2023-01-22 13:19:18+00:00,00150363,20,126,126,0.000000,0.000000,0.000000
7,2023-01-24 06:20:58+00:00,00150363,24,145,168,0.083333,0.013793,0.011905
8,2023-01-24 10:58:26+00:00,00150363,20,146,172,0.100000,0.013699,0.011628
9,2023-01-10 01:09:12+00:00,01038865,0,0,0,0.000000,0.000000,0.000000


### Define customer and terminal-related features for _real-time_ feature engineering

To make more accurate predictions, you can also create real-time features to inspect, for example, the most recent minutes of activity for both customers and terminals.

In order to have these features available for training and inference, we first need to make sure they are defined in a BigQuery table as placeholders to be ingested into Vertex AI Feature Store.

In the query below, you initialize two sets of features for real-time feature engineering: 

1. Customer features which describes the spending behaviour of customer within 15, 30 and 60 minutes time windows using number of transactions and average amount spent in dollars ($)

2. Terminal features which describes the risk of a given terminal to be exposed to fraudulent transactions within 15, 30 and 60 minutes using average number of fraudulent transactions in dollars ($) and the number of transactions. 

To do so, you will:

- Add one column for each real time feature
- Set 0 as default values for each of them
- Update all real-time columns with default values

Then you will create the actual values for real-time feature engineering in the next `03_feature_engineering_streaming.ipynb` notebook. 

#### Define the query to initialize the real-time features.

##### Customer feature table

Customer table SQL query string:

In [14]:
initiate_real_time_customer_features_query = f"""
ALTER TABLE `{CUSTOMERS_BQ_TABLE_URI}`
ADD COLUMN customer_id_nb_tx_15min_window INT64,
ADD COLUMN customer_id_nb_tx_30min_window INT64,
ADD COLUMN customer_id_nb_tx_60min_window INT64,
ADD COLUMN customer_id_avg_amount_15min_window FLOAT64,
ADD COLUMN customer_id_avg_amount_30min_window FLOAT64,
ADD COLUMN customer_id_avg_amount_60min_window FLOAT64;

ALTER TABLE `{CUSTOMERS_BQ_TABLE_URI}`
ALTER COLUMN customer_id_nb_tx_15min_window SET DEFAULT 0,
ALTER COLUMN customer_id_nb_tx_30min_window SET DEFAULT 0,
ALTER COLUMN customer_id_nb_tx_60min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_15min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_30min_window SET DEFAULT 0,
ALTER COLUMN customer_id_avg_amount_60min_window SET DEFAULT 0;

UPDATE `{CUSTOMERS_BQ_TABLE_URI}`
SET customer_id_nb_tx_15min_window = 0,
    customer_id_nb_tx_30min_window  = 0,
    customer_id_nb_tx_60min_window  = 0, 
    customer_id_avg_amount_15min_window = 0,
    customer_id_avg_amount_30min_window  = 0,
    customer_id_avg_amount_60min_window  = 0
WHERE TRUE; 
"""

In [15]:
initiate_real_time_terminal_features_query = f"""
ALTER TABLE `{TERMINALS_BQ_TABLE_URI}`
ADD COLUMN terminal_id_nb_tx_15min_window INT64,
ADD COLUMN terminal_id_nb_tx_30min_window INT64,
ADD COLUMN terminal_id_nb_tx_60min_window INT64,
ADD COLUMN terminal_id_avg_amount_15min_window FLOAT64,
ADD COLUMN terminal_id_avg_amount_30min_window FLOAT64,
ADD COLUMN terminal_id_avg_amount_60min_window FLOAT64;

ALTER TABLE `{TERMINALS_BQ_TABLE_URI}`
ALTER COLUMN terminal_id_nb_tx_15min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_nb_tx_30min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_nb_tx_60min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_15min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_30min_window SET DEFAULT 0,
ALTER COLUMN terminal_id_avg_amount_60min_window SET DEFAULT 0;

UPDATE `{TERMINALS_BQ_TABLE_URI}`
SET terminal_id_nb_tx_15min_window = 0,
    terminal_id_nb_tx_30min_window  = 0,
    terminal_id_nb_tx_60min_window  = 0,
    terminal_id_avg_amount_15min_window = 0,
    terminal_id_avg_amount_30min_window = 0,
    terminal_id_avg_amount_60min_window  = 0
WHERE TRUE; 
"""

#### Run the query above to initialize the real-time features.

In [16]:
for query in [initiate_real_time_customer_features_query, initiate_real_time_terminal_features_query]:
    run_bq_query(query)

Finished job_id: b224f5dd-2151-42bd-9aba-31433ae66f72
Finished job_id: 4a0c6f9c-6f7b-4231-881a-274fd8d282c2


#### Inspect BigQuery features tables

In [17]:
run_bq_query(
    f"SELECT * FROM `{CUSTOMERS_BQ_TABLE_URI}` LIMIT 5", show=True
)

Finished job_id: 17d2455e-b8b4-43b2-8e43-a2c0f07fbde5


,feature_ts,customer_id,customer_id_nb_tx_1day_window,customer_id_nb_tx_7day_window,customer_id_nb_tx_14day_window,customer_id_avg_amount_1day_window,customer_id_avg_amount_7day_window,customer_id_avg_amount_14day_window,customer_id_nb_tx_15min_window,customer_id_nb_tx_30min_window,customer_id_nb_tx_60min_window,customer_id_avg_amount_15min_window,customer_id_avg_amount_30min_window,customer_id_avg_amount_60min_window
0,2023-01-09 05:07:13+00:00,4720361778782431,1,1,1,26.47,26.47,26.47,0,0,0,0.0,0.0,0.0
1,2023-01-09 17:40:09+00:00,8213393238302219,1,1,1,19.59,19.59,19.59,0,0,0,0.0,0.0,0.0
2,2023-01-09 01:48:02+00:00,9915614618490038,1,1,1,2.85,2.85,2.85,0,0,0,0.0,0.0,0.0
3,2023-01-09 01:22:33+00:00,9156124705830581,1,1,1,22.08,22.08,22.08,0,0,0,0.0,0.0,0.0
4,2023-01-20 21:22:59+00:00,2103339204148833,1,1,1,18.13,18.13,18.13,0,0,0,0.0,0.0,0.0


In [18]:
run_bq_query(
    f"SELECT * FROM `{TERMINALS_BQ_TABLE_URI}` LIMIT 5", show=True
)

Finished job_id: 5cba9fd9-90c4-462f-bd22-474d2960a45e


,feature_ts,terminal_id,terminal_id_nb_tx_1day_window,terminal_id_nb_tx_7day_window,terminal_id_nb_tx_14day_window,terminal_id_risk_1day_window,terminal_id_risk_7day_window,terminal_id_risk_14day_window,terminal_id_nb_tx_15min_window,terminal_id_nb_tx_30min_window,terminal_id_nb_tx_60min_window,terminal_id_avg_amount_15min_window,terminal_id_avg_amount_30min_window,terminal_id_avg_amount_60min_window
0,2023-01-09 05:29:21+00:00,13010235,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
1,2023-01-15 19:24:56+00:00,86234185,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
2,2023-01-14 05:40:24+00:00,26843539,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
3,2023-01-14 22:18:05+00:00,28148204,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0
4,2023-01-10 21:17:01+00:00,31489695,0,0,0,0.0,0.0,0.0,0,0,0,0.0,0.0,0.0


Let's look at the final schema of the features table:

In [19]:
run_bq_query(f"""
SELECT column_name, data_type
FROM tx.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{CUSTOMERS_TABLE_NAME}'
""", show=True)

Finished job_id: de9f7c2c-e8da-432e-a365-352137ea8b0f


,column_name,data_type
0,feature_ts,TIMESTAMP
1,customer_id,STRING
2,customer_id_nb_tx_1day_window,INT64
3,customer_id_nb_tx_7day_window,INT64
4,customer_id_nb_tx_14day_window,INT64
5,customer_id_avg_amount_1day_window,FLOAT64
6,customer_id_avg_amount_7day_window,FLOAT64
7,customer_id_avg_amount_14day_window,FLOAT64
8,customer_id_nb_tx_15min_window,INT64
9,customer_id_nb_tx_30min_window,INT64


In [20]:
run_bq_query(f"""
SELECT column_name, data_type
FROM tx.INFORMATION_SCHEMA.COLUMNS
WHERE table_name = '{TERMINALS_TABLE_NAME}'
""", show=True)

Finished job_id: 3851d703-205c-4db7-8712-b9ae5c47c9bf


,column_name,data_type
0,feature_ts,TIMESTAMP
1,terminal_id,STRING
2,terminal_id_nb_tx_1day_window,INT64
3,terminal_id_nb_tx_7day_window,INT64
4,terminal_id_nb_tx_14day_window,INT64
5,terminal_id_risk_1day_window,FLOAT64
6,terminal_id_risk_7day_window,FLOAT64
7,terminal_id_risk_14day_window,FLOAT64
8,terminal_id_nb_tx_15min_window,INT64
9,terminal_id_nb_tx_30min_window,INT64


## Feature store for feature management

### What is a feature store?

The features generated are great examples of features that we can store the [Vertex AI Feature Store](https://cloud.google.com/vertex-ai/docs/featurestore). This is because:

- The features are needed for real-time prediction
- feature values in a feature store can be used for both training and serving
- if needed, features can be shared with other use cases beyond fraud detection

Vertex AI Feature Store provides a centralized repository for organizing, storing, and serving ML features. Using a central featurestore enables an organization to efficiently share, discover, and re-use ML features at scale, which can increase the velocity of developing and deploying new ML applications.

### Why would you like to set up it?

So far you've built and stored features in BigQuery. 

Now, in order to predict fraud, you want to serve those features in real-time with millisecond scale latency. In particular, when the ML gateway receives a prediction request for a specific transaction (including customer, terminal, and transaction ids), the system needs to fetch the features related to that transaction and pass them as inputs to the model for online prediction. As you can imagine, an analytical data warehouse such as BigQuery is not able to provide low-latency near real-time read operations. 

Vertex AI Feature Store provides a managed service for low latency scalable feature serving. It also provides a centralized feature repository with easy APIs to search and discover features, as well as feature monitoring capabilities to track drift and other quality issues. 

Vertex AI Feature Store uses a time series data model to store a series of values for features, which enables Vertex AI Feature Store to maintain feature values as they change over time and to support point-in-time queries of feature values. Feature Store organizes resources hierarchically (`Featurestore -> EntityType -> Feature`) in the following order: 

- **Featurestore**: the resource to contains entities and features.
    - **EntityType**: under a Featurestore, an EntityType describes an minimal data entry.
        - **Feature**: under an EntityType, a feature is an attribute of the EntityType. 


You must create these resources before you can ingest data into a Feature Store. 

In the follow section, you will use create a feature store using Vertex AI Feature Store, and ingest data into it to be used later for training and model inference.

### Initialize Vertex AI SDK

Initialize the Vertex AI SDK to get access to Vertex AI services programmatically. 

In [21]:
vertex_ai.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

### Create featurestore, `fraudfinder_<ID>`

A featurestore is the top-level container for entity types, features, and feature values. Typically, an organization creates one shared featurestore for feature ingestion, serving, and sharing across all teams in the organization.

Below you create a `featurestore` resources with different labels. 

In [22]:
# Try to create a new featurestore resource
ff_feature_store = Featurestore.create(
    featurestore_id=f"{FEATURESTORE_ID}",
    online_store_fixed_node_count=ONLINE_STORAGE_NODES,
    labels={"team": "cymbal_bank", "app": "fraudfinder"},
    sync=True,
)

Creating Featurestore
Create Featurestore backing LRO: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/operations/2793732396183191552
Featurestore created. Resource name: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj
To use this Featurestore in another session:
featurestore = aiplatform.Featurestore('projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj')


### Create the main entity types and their features

An entity type is a collection of semantically related features. You define your own entity types, based on the concepts that are relevant to your use case. 

In this case, you create `customer` and `transaction` entity types. 

#### Create the ```customer``` entity type 

In [23]:
customer_entity_type = ff_feature_store.create_entity_type(
    entity_type_id=CUSTOMER_ENTITY_ID, description="Customer Entity", sync=True
)

Creating EntityType
Create EntityType backing LRO: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/customer/operations/1149918532192960512
EntityType created. Resource name: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/customer
To use this EntityType in another session:
entity_type = aiplatform.EntityType('projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/customer')


#### Create features of the ```customer``` entity type

In [24]:
customer_feature_configs = {
    "customer_id_nb_tx_1day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last day",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_7day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 7 days",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_14day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 14 days",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_1day_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last day",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_7day_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 7 days",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_14day_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 14 days",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_15min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_30min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_nb_tx_60min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the customer in the last 60 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_15min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_30min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "customer_id_avg_amount_60min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 60 minutes",
        "labels": {"status": "passed"},
    },
}

In [25]:
customer_feature_ids = customer_entity_type.batch_create_features(
    feature_configs=customer_feature_configs, sync=True
)

Batch creating features EntityType entityType: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/customer
Batch create Features EntityType entityType backing LRO: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/operations/1199458128094035968
EntityType entityType Batch created features. Resource name: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/customer


#### Create the ```terminal``` entity type

In [26]:
terminal_entity_type = ff_feature_store.create_entity_type(
    entity_type_id=TERMINAL_ENTITY_ID, description="Terminal Entity", sync=True
)

Creating EntityType
Create EntityType backing LRO: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/terminal/operations/6761403667896598528
EntityType created. Resource name: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/terminal
To use this EntityType in another session:
entity_type = aiplatform.EntityType('projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/terminal')


#### Create features of the ```terminal``` entity type

In [27]:
terminal_feature_configs = {
    "terminal_id_nb_tx_1day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last day",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_7day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the 7 days",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_14day_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the 14 days",
        "labels": {"status": "passed"},
    },
    "terminal_id_risk_1day_window": {
        "value_type": "DOUBLE",
        "description": "Risk score calculated average number of frauds on the terminal in the last day",
        "labels": {"status": "passed"},
    },
    "terminal_id_risk_7day_window": {
        "value_type": "DOUBLE",
        "description": "Risk score calculated average number of frauds on the terminal in the last 7 days",
        "labels": {"status": "passed"},
    },
    "terminal_id_risk_14day_window": {
        "value_type": "DOUBLE",
        "description": "Risk score calculated average number of frauds on the terminal in the last 14 day",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_15min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_30min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_nb_tx_60min_window": {
        "value_type": "INT64",
        "description": "Number of transactions by the terminal in the last 60 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_avg_amount_15min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 15 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_avg_amount_30min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 30 minutes",
        "labels": {"status": "passed"},
    },
    "terminal_id_avg_amount_60min_window": {
        "value_type": "DOUBLE",
        "description": "Average spending amount in the last 60 minutes",
        "labels": {"status": "passed"},
    },
}

In [28]:
terminal_feature_ids = terminal_entity_type.batch_create_features(
    feature_configs=terminal_feature_configs, sync=True
)

Batch creating features EntityType entityType: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/terminal
Batch create Features EntityType entityType backing LRO: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/operations/9218117264627204096
EntityType entityType Batch created features. Resource name: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/terminal


### Inspect your feature store in the Vertex AI console

You can also inspect your feature store in the [Vertex AI Feature Store console](https://console.cloud.google.com/vertex-ai/features)

### Ingest feature values in Vertex AI Feature Store

Now we need to ingest the actual feature values you created in BigQuery into the Vertex AI Feature Store.

To ingest features values in Vertex AI Feature Store, you need to check the following requirements related to **Source Data format and Layout**:

- Features values have to [be stored](https://cloud.google.com/vertex-ai/docs/featurestore/source-data) in BigQuery tables or Avro and CSV files on Google Cloud Storage.
- Each imported feature entity *must* have an ID.
- Each feature entity can *optionally* have a timestamp, to specifying when the feature values are generated.

In the following queries, you ingest feature values from those BigQuery tables into Vertex AI Feature Store.

#### Ingest customer feature values into `customers` entity in Vertex AI Feature Store 

In the following section, you will import customer feature values into your feature store.

In [29]:
CUSTOMERS_FEATURES_IDS = [
    feature.name for feature in customer_feature_ids.list_features()
]
CUSTOMER_BQ_SOURCE_URI = f"bq://{CUSTOMERS_BQ_TABLE_URI}"
CUSTOMER_ENTITY_ID_FIELD = "customer_id"

In [30]:
customer_entity_type.ingest_from_bq(
    feature_ids=CUSTOMERS_FEATURES_IDS,
    feature_time=FEATURE_TIME,
    bq_source_uri=CUSTOMER_BQ_SOURCE_URI,
    entity_id_field=CUSTOMER_ENTITY_ID_FIELD,
    disable_online_serving=False,
    worker_count=10,
    sync=True,
)

Importing EntityType feature values: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/customer
Import EntityType feature values backing LRO: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/customer/operations/8227325346605694976
EntityType feature values imported. Resource name: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/customer


resource name: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/customer

#### Monitor the `customer` features ingestion job in the console.


You can go to the [Feature Store Console](https://console.cloud.google.com/vertex-ai/ingestion-jobs) to view your ingestion job. 

#### Ingest terminal feature values into `terminal` entity in Vertex AI Feature Store 

In the following section, you will import terminal feature values into your feature store.

In [31]:
TERMINAL_ENTITY_ID = "terminal"
TERMINALS_FEATURES_IDS = [
    feature.name for feature in terminal_feature_ids.list_features()
]
TERMINALS_BQ_SOURCE_URI = f"bq://{TERMINALS_BQ_TABLE_URI}"
TERMINALS_ENTITY_ID_FIELD = "terminal_id"

In [32]:
terminal_entity_type.ingest_from_bq(
    feature_ids=TERMINALS_FEATURES_IDS,
    feature_time=FEATURE_TIME,
    bq_source_uri=TERMINALS_BQ_SOURCE_URI,
    entity_id_field=TERMINALS_ENTITY_ID_FIELD,
    disable_online_serving=False,
    worker_count=10,
    sync=True,
)

Importing EntityType feature values: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/terminal
Import EntityType feature values backing LRO: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/terminal/operations/787378762189635584
EntityType feature values imported. Resource name: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/terminal


resource name: projects/547090333450/locations/us-central1/featurestores/fraudfinder_9selj/entityTypes/terminal

#### Monitor the ingestion jobs in the console.

The ingestion jobs you just created run asynchronously and they should take several minutes to complete. Please monitoring them in the [console](https://console.cloud.google.com/vertex-ai/ingestion-jobs).


### Search for feature values 
In this section, you'll run a search query on your feature store to validate that some data was ingested, as expected.

In [33]:
customer_aggregated_features = customer_entity_type.read(
    entity_ids=["5830444124423549", "5469689693941771", "1361459972478769"],
    feature_ids=CUSTOMERS_FEATURES_IDS,
)

In [34]:
customer_aggregated_features

,entity_id,customer_id_avg_amount_15min_window,customer_id_nb_tx_7day_window,customer_id_avg_amount_14day_window,customer_id_avg_amount_60min_window,customer_id_avg_amount_1day_window,customer_id_nb_tx_30min_window,customer_id_avg_amount_30min_window,customer_id_nb_tx_14day_window,customer_id_nb_tx_15min_window,customer_id_avg_amount_7day_window,customer_id_nb_tx_1day_window,customer_id_nb_tx_60min_window
0,1361459972478769,0.0,16,51.157407,0.0,66.07,0,0.0,27,0,50.871875,1,0
1,5469689693941771,0.0,21,46.235116,0.0,46.70,0,0.0,43,0,46.647143,2,0
2,5830444124423549,0.0,23,54.548667,0.0,58.54,0,0.0,30,0,54.446957,2,0


### END

Now you can go to the next notebook `03_feature_engineering_streaming.ipynb`

## (DO NOT RUN) Clean up

In [35]:
# ff_feature_store.delete(sync=True, force=True)